# Titanic Competitions Kaggle (DONE)

In [175]:
data = pd.read_csv("data/train.csv")
data

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [176]:
# data = data.drop(columns=["PassengerId", "Name", "Ticket", "Cabin"], axis=1)
data.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [177]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def evaluate_preds(y_true, y_preds):
    """
    Performs evaluation 
    """

    accuracy = accuracy_score(y_true, y_preds)
    precision = precision_score(y_true, y_preds)
    recall = recall_score(y_true, y_preds)
    f1 = f1_score(y_true, y_preds)
    metric_dict = {
        "accuracy" : round(accuracy, 2),
        "precision" : round(precision, 2),
        "recall" : round(recall, 2),
        "f1" : round(f1, 2)
    }
    print(f"Accuracy: {accuracy * 100 :.2f}%\nPrecision : {precision :.2f}\nRecall : {recall :.2f}\nF1 : {f1 :.2f}")
    return metric_dict

In [178]:
number_features = ["Pclass", "Age", "SibSp", "Parch", "Fare"]
number_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="median")),
    ("standarScal", StandardScaler())
])

category_features = ["Sex", "Embarked"]
category_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("oneHot", OneHotEncoder(handle_unknown="ignore"))
])

preprocessing = ColumnTransformer(transformers=[
    ("num", number_transformer, number_features),
    ("cat", category_transformer, category_features)
])

model = Pipeline(steps=[
    ("preprocessing", preprocessing),
    ("model", RandomForestClassifier())
])

np.random.seed(42)

X = data.drop("Survived", axis=1)
y = data["Survived"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

model.fit(X_train, y_train)
model.score(X_test, y_test)

0.8044692737430168

In [179]:
y_preds = model.predict(X_test)
metrics = evaluate_preds(y_test, y_preds)

Accuracy: 80.45%
Precision : 0.77
Recall : 0.76
F1 : 0.76


In [180]:
from sklearn.model_selection import GridSearchCV

pipe_grid = {
    "model__n_estimators": [50, 100, 200, 300, 500],
    "model__min_samples_leaf": [1, 3, 5],
    "model__min_samples_split": [2, 5, 10],
    "model__max_depth": [3, 5, 10, None],
    "model__max_features": ["sqrt", "log2"]
}

gs_clf = GridSearchCV(estimator=model, param_grid=pipe_grid, cv=5, verbose=2)
gs_clf.fit(X_train, y_train)

Fitting 5 folds for each of 360 candidates, totalling 1800 fits
[CV] END model__max_depth=3, model__max_features=sqrt, model__min_samples_leaf=1, model__min_samples_split=2, model__n_estimators=50; total time=   0.2s
[CV] END model__max_depth=3, model__max_features=sqrt, model__min_samples_leaf=1, model__min_samples_split=2, model__n_estimators=50; total time=   0.1s
[CV] END model__max_depth=3, model__max_features=sqrt, model__min_samples_leaf=1, model__min_samples_split=2, model__n_estimators=50; total time=   0.1s
[CV] END model__max_depth=3, model__max_features=sqrt, model__min_samples_leaf=1, model__min_samples_split=2, model__n_estimators=50; total time=   0.1s
[CV] END model__max_depth=3, model__max_features=sqrt, model__min_samples_leaf=1, model__min_samples_split=2, model__n_estimators=50; total time=   0.1s
[CV] END model__max_depth=3, model__max_features=sqrt, model__min_samples_leaf=1, model__min_samples_split=2, model__n_estimators=100; total time=   0.3s
[CV] END model__m

C:\Users\oktax\miniconda3\envs\work-env\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessing',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='median')),
                                                                                         ('standarScal',
                                                                                          StandardScaler())]),
                                                                         ['Pclass',
                                                                          'Age',
                                                                          'SibSp',
                                                                          'Parch',
                                                                          'Fare']),
                                                                        ('cat',
                                                                         Pipeline(steps=[('imputer',
                                                                                          SimpleImputer(strategy='most_frequent')),
                                                                                         ('oneHot',
                                                                                          OneHotEncoder(handle_unknown='ignore'))]),
                                                                         ['Sex',
                                                                          'Embarked'])])),
                                       ('model', RandomForestClassifier())]),
             param_grid={'model__max_depth': [3, 5, 10, None],
                         'model__max_features': ['sqrt', 'log2'],
                         'model__min_samples_leaf': [1, 3, 5],
                         'model__min_samples_split': [2, 5, 10],
                         'model__n_estimators': [50, 100, 200, 300, 500]},
             verbose=2)

In [187]:
y_preds_2 = gs_clf.predict(X_test)
metric_2 = evaluate_preds(y_test, y_preds_2)

Accuracy: 80.45%
Precision : 0.80
Recall : 0.70
F1 : 0.75


In [190]:
test_df = pd.read_csv("data/test.csv")
X_test_final = test_df[number_features + category_features]

y_pred = gs_clf.predict(X_test_final)

submission = pd.DataFrame({
    "PassengerId": test_df["PassengerId"],
    "Survived": y_pred
})

submission.to_csv("submission.csv", index=False)
